### Objective-

- We have $1000 left in our marketing budget and it won't roll over next year. We can use this to convert some physical store customers into online.
- We have to make sure though that we aren't stealing any customer from our brick and mortor store counterparts(Physical stores). We have been asked to use our judgement to identify the customers whom we can send the coupons to, so that they can shop online.

So let's break down our strategy as communicated by the client:
- Choose only in store customers
- Focus on the custmores that have been churned


In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_table("rfm_xmas19.txt", delimiter=",")

In [3]:
data

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78
...,...,...,...
124995,FM8433,2016-03-26,64
124996,FM7232,2019-05-19,38
124997,FM8731,2019-08-28,42
124998,FM8133,2018-09-14,13


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [5]:
data['trans_date'] = pd.to_datetime(data['trans_date'])

In [6]:
data['trans_date'].max()

Timestamp('2019-12-16 00:00:00')

- We want churned customers or customers who stopped shopping. We have a transaction date column for each customer. We can easily shortlist them based on that. Given that most our physical customers buy groceries, any customer who hasn't shopped in over three months in our store can be safely classified as a customer who has churned

- Now since we will be working with different dates, let's assume that the max date (2019-12-16) is the day we recieved the data. So any customer whose latest transaction date is before October 16, 2019 can be classified as churned.

In [7]:
cut_off = dt.datetime(2019,9,16)

In [8]:
cut_off

datetime.datetime(2019, 9, 16, 0, 0)

We have multiple rows for each customer. We can group them together and use their latest transaction date as a point of reference. 

In [9]:
grouped_object = data.groupby('customer_id')

In [10]:
cust_id_grouped = grouped_object['trans_date'].max()

In [11]:
cust_id_grouped

customer_id
FM1112   2019-10-14
FM1113   2019-11-09
FM1114   2019-11-12
FM1115   2019-12-05
FM1116   2019-05-25
            ...    
FM8996   2019-09-09
FM8997   2019-03-28
FM8998   2019-09-22
FM8999   2019-04-02
FM9000   2019-11-28
Name: trans_date, Length: 6889, dtype: datetime64[ns]

Let us create a new dataframe that will contain the neccessary columns for our analysis.

In [12]:
customer_analysis = pd.DataFrame(cust_id_grouped)

In [13]:
customer_analysis

,trans_date
customer_id,
FM1112,2019-10-14
FM1113,2019-11-09
FM1114,2019-11-12
FM1115,2019-12-05
FM1116,2019-05-25
...,...
FM8996,2019-09-09
FM8997,2019-03-28
FM8998,2019-09-22


- Let's now add a column named 'churn' that will classify as 1 for a customer who is considered churned and 0 otherwise

In [14]:
customer_analysis['churn'] = 0

In [15]:
customer_analysis.loc[customer_analysis['trans_date'] <= cut_off, 'churn'] = 1

In [16]:
customer_analysis

,trans_date,churn
customer_id,,
FM1112,2019-10-14,0
FM1113,2019-11-09,0
FM1114,2019-11-12,0
FM1115,2019-12-05,0
FM1116,2019-05-25,1
...,...,...
FM8996,2019-09-09,1
FM8997,2019-03-28,1
FM8998,2019-09-22,0


- We can now filter and choose only those customers who are classified as churned.
- Let us add more columns to the data first such as number of transactions and total amount spent before filtering. 

In [17]:
grouped_object.size()

customer_id
FM1112    15
FM1113    20
FM1114    19
FM1115    22
FM1116    13
          ..
FM8996    13
FM8997    14
FM8998    13
FM8999    12
FM9000    13
Length: 6889, dtype: int64

In [18]:
customer_analysis['num_of_transactions'] = grouped_object.size()

In [19]:
grouped_object['tran_amount'].sum()

customer_id
FM1112    1012
FM1113    1490
FM1114    1432
FM1115    1659
FM1116     857
          ... 
FM8996     582
FM8997     543
FM8998     624
FM8999     383
FM9000     533
Name: tran_amount, Length: 6889, dtype: int64

In [20]:
customer_analysis['Total_tran_amount'] = grouped_object['tran_amount'].sum()

In [21]:
customer_analysis

,trans_date,churn,num_of_transactions,Total_tran_amount
customer_id,,,,
FM1112,2019-10-14,0,15,1012
FM1113,2019-11-09,0,20,1490
FM1114,2019-11-12,0,19,1432
FM1115,2019-12-05,0,22,1659
FM1116,2019-05-25,1,13,857
...,...,...,...,...
FM8996,2019-09-09,1,13,582
FM8997,2019-03-28,1,14,543
FM8998,2019-09-22,0,13,624


- Let us now filter our data to only select those customers that are churned. 

In [22]:
churned_customers = customer_analysis[customer_analysis['churn']==1]

In [23]:
churned_customers.head()

,trans_date,churn,num_of_transactions,Total_tran_amount
customer_id,,,,
FM1116,2019-05-25,1,13,857
FM1117,2019-04-02,1,17,1185
FM1123,2019-08-27,1,19,1331
FM1126,2019-06-18,1,19,1165
FM1129,2019-08-30,1,12,853


In [24]:
churned_customers.shape

(2221, 4)

In [25]:
churned_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2221 entries, FM1116 to FM8999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   trans_date           2221 non-null   datetime64[ns]
 1   churn                2221 non-null   int64         
 2   num_of_transactions  2221 non-null   int64         
 3   Total_tran_amount    2221 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 86.8+ KB


In [26]:
churned_customers['trans_date'].min()

Timestamp('2017-08-09 00:00:00')

Here is a predicament. There are a few steps we can take and the cons associated with them:
- We can filter to choose only those customers that have stopped buying from in the year 2019. Considering  the fact that we only have a budget of $1000, we cannot cater to all of the customers in our dataset. The downside of this approach is that we will filter out customers who might have had a high a average cart value in the previous years. We need to ensure maximum return from our advertising spend and customers with a high cart value are better candidate.
  
- We can sort the results based on total spend or number of transactions. But that leaves us with another question. Is the number of transactions a better indicator of a good customer or total spend? A customer can have a high cart value but have a low number of transactions. You can't classify these customers as regulars.
  
- The third approach can be to assign a scoring system. Taking both spend and number of transactions into consideration and then scoring based on the combination of both the columns. This approach by far seems the most reasonable. We have to be careful, though and ensure we come up with a scoring system that isn't biased towards one column or the other.

- We have decided that we will filter the most recent customers and then use a scoring system to filter further and target those customers with a high score.
- We will use the weighted sum model. In the nutshell, we are assigning both our columns equal weights.
- **So the formula would be (1/2 x number of transactions) + (1/2 x total spend)**

In [27]:
churned_customers.describe()

,trans_date,churn,num_of_transactions,Total_tran_amount
count,2221,2221.0,2221.000000,2221.000000
mean,2019-06-22 07:30:36.470058496,1.0,16.515083,1040.498874
min,2017-08-09 00:00:00,1.0,4.000000,149.000000
25%,2019-05-23 00:00:00,1.0,13.000000,627.000000
50%,2019-07-19 00:00:00,1.0,16.000000,1067.000000
75%,2019-08-22 00:00:00,1.0,20.000000,1374.000000
max,2019-09-16 00:00:00,1.0,36.000000,2513.000000
std,NaN,0.0,5.096048,456.767227


- It is to be noted that the range for number of transactions is from 4-36 while the range for amount spent is 149-2513. This could be a problem. If we don't scale our data before applying the weighted sum model, there is bound to be some bias towards total spend as compared to num of transactions, as we have resorted to assign them equal weights.
- That is the reason we will first have to scale these two columns. Let's use MIN-MAX SCALING here.

In [28]:
churned_customers['scaled_num_transactions'] = (churned_customers['num_of_transactions'] - churned_customers['num_of_transactions'].min())/(churned_customers['num_of_transactions'].max() - churned_customers['num_of_transactions'].min())

In [29]:
churned_customers.head()

,trans_date,churn,num_of_transactions,Total_tran_amount,scaled_num_transactions
customer_id,,,,,
FM1116,2019-05-25,1,13,857,0.28125
FM1117,2019-04-02,1,17,1185,0.40625
FM1123,2019-08-27,1,19,1331,0.46875
FM1126,2019-06-18,1,19,1165,0.46875
FM1129,2019-08-30,1,12,853,0.25000


In [30]:
churned_customers['scaled_tran_amount'] = (churned_customers['Total_tran_amount'] - churned_customers['Total_tran_amount'].min())/(churned_customers['Total_tran_amount'].max() - churned_customers['Total_tran_amount'].min()) 

In [31]:
churned_customers

,trans_date,churn,num_of_transactions,Total_tran_amount,scaled_num_transactions,scaled_tran_amount
customer_id,,,,,,
FM1116,2019-05-25,1,13,857,0.28125,0.299492
FM1117,2019-04-02,1,17,1185,0.40625,0.438240
FM1123,2019-08-27,1,19,1331,0.46875,0.500000
FM1126,2019-06-18,1,19,1165,0.46875,0.429780
FM1129,2019-08-30,1,12,853,0.25000,0.297800
...,...,...,...,...,...,...
FM8990,2019-09-12,1,13,613,0.28125,0.196277
FM8992,2019-06-28,1,9,277,0.15625,0.054146
FM8996,2019-09-09,1,13,582,0.28125,0.183164


- Let's add the scoring column and then clean up our dataframe. 

In [32]:
churned_customers['score'] = ((0.5 * churned_customers['scaled_num_transactions']) + (0.5 * churned_customers['scaled_tran_amount'])) * 100

In [33]:
churned_customers.head()

,trans_date,churn,num_of_transactions,Total_tran_amount,scaled_num_transactions,scaled_tran_amount,score
customer_id,,,,,,,
FM1116,2019-05-25,1,13,857,0.28125,0.299492,29.037119
FM1117,2019-04-02,1,17,1185,0.40625,0.438240,42.224514
FM1123,2019-08-27,1,19,1331,0.46875,0.500000,48.437500
FM1126,2019-06-18,1,19,1165,0.46875,0.429780,44.926502
FM1129,2019-08-30,1,12,853,0.25000,0.297800,27.390017


In [34]:
cut_off2 = dt.datetime(2019,1,1)

In [35]:
final_cleaned_data = churned_customers[churned_customers['trans_date']>=cut_off2]

In [36]:
final_cleaned_data = final_cleaned_data.sort_values(by = ['trans_date'])

In [37]:
final_cleaned_data

,trans_date,churn,num_of_transactions,Total_tran_amount,scaled_num_transactions,scaled_tran_amount,score
customer_id,,,,,,,
FM8730,2019-01-01,1,13,692,0.28125,0.229695,25.547272
FM2730,2019-01-01,1,16,1186,0.37500,0.438663,40.683164
FM7939,2019-01-02,1,11,498,0.21875,0.147631,18.319057
FM7839,2019-01-02,1,11,492,0.21875,0.145093,18.192153
FM7719,2019-01-03,1,13,657,0.28125,0.214890,24.807001
...,...,...,...,...,...,...,...
FM1737,2019-09-16,1,23,1737,0.59375,0.671743,63.274640
FM7808,2019-09-16,1,10,437,0.18750,0.121827,15.466371
FM4456,2019-09-16,1,22,1701,0.56250,0.656514,60.950719


In [38]:
final_cleaned_data = final_cleaned_data.drop(columns=['trans_date','churn','num_of_transactions','Total_tran_amount'])

In [39]:
final_cleaned_data.head()

,scaled_num_transactions,scaled_tran_amount,score
customer_id,,,
FM8730,0.28125,0.229695,25.547272
FM2730,0.37500,0.438663,40.683164
FM7939,0.21875,0.147631,18.319057
FM7839,0.21875,0.145093,18.192153
FM7719,0.28125,0.214890,24.807001


In [40]:
final_cleaned_data.sort_values(['score'], ascending=False, inplace=True)

In [41]:
final_cleaned_data.head()

,scaled_num_transactions,scaled_tran_amount,score
customer_id,,,
FM3799,1.00000,1.000000,100.000000
FM3805,0.96875,0.974619,97.168464
FM4074,0.93750,0.978426,95.796320
FM1215,0.96875,0.936125,95.243761
FM2620,0.96875,0.935279,95.201459


In [42]:
final_cleaned_data.shape

(2095, 3)

- The final task is that how do we go about distributing the coupons. It's been left at our discretion to decide what the coupon amount should be and which customers to distribute it to. It is rather tricky to choose the coupon amount. If we choose too high an amount, we pretty much end up giving away the money and the whole exercise ends up incurring a loss to the firm. Choose a coupon value too low and it might not entice the custmors to go online and shop at the website.

- It was wise to get in touch with the marketing department and ask them what according to their experienced opinion is the best discount rate. They told its 30% 
- So, we can now go ahead calculate the coupon price

In [43]:
coupon_price = data['tran_amount'].mean() * 0.30
coupon_price

19.4975736

- Now we only have $1000 to spare. To decide the number of customers that can recieve the coupons, we can simply divide 1000 by the coupon value as shown.

In [44]:
num_of_customers = 1000/19
num_of_customers

52.63157894736842

- Rounding off, we can choose 52 customers to distribute $19 coupon among them

In [45]:
final_cust_list = final_cleaned_data.head(52)
final_cust_list

,scaled_num_transactions,scaled_tran_amount,score
customer_id,,,
FM3799,1.00000,1.000000,100.000000
FM3805,0.96875,0.974619,97.168464
FM4074,0.93750,0.978426,95.796320
FM1215,0.96875,0.936125,95.243761
FM2620,0.96875,0.935279,95.201459
FM1580,0.90625,0.922166,91.420791
FM2951,0.87500,0.944585,90.979272
FM5868,0.84375,0.892978,86.836400
FM1695,0.87500,0.856599,86.579949


In [46]:
final_cust_list.drop(columns=['scaled_num_transactions','scaled_tran_amount','score'], inplace=True)